# Split merged trajectories

Split a trajectory which is actually a merge of independent trajectories back to the original pieces

Run an RMSD analysis to guess where the pieces are

In [2]:
import mdtraj as mdt

input_structure_filename = 'nsp14_dry_segname.psf'
input_trajectory_filename = 'mdall_namd_nodummies.dcd'

Load the trajectory

In [ ]:
trajectory = mdt.load(input_trajectory_filename, top=input_structure_filename)

Run a RMSD analysis

In [4]:
rmsd_data = mdt.rmsd(trajectory, trajectory, 0)

Find sudden jumps in RMSD values

In [20]:
sudden_jump_cutoff = 0.2
sudden_jumps = []
previous_value = 0
for i, value in enumerate(rmsd_data):
    diff = value - previous_value
    if abs(diff) > sudden_jump_cutoff:
        print('Sudden jump in frame ' + str(i))
        sudden_jumps.append(i)
    previous_value = value

Sudden jump in frame 10000
Sudden jump in frame 16000


In [28]:
cut_points = [ 0, *sudden_jumps, len(rmsd_data) ]
for i in range(len(cut_points) -1):
    start = cut_points[i]
    end = cut_points[i+1]
    trajectory_split = trajectory[start:end]
    split_filename = 'split_' + str(i) + '.dcd'
    trajectory_split.save(split_filename)

### Show the RMSD plot

In [17]:
import plotly.graph_objs as go

# Set the vertical lines to show the cuts
shapes = []
for jump in sudden_jumps:
    max_value = max(rmsd_data)
    min_value = min(rmsd_data)
    shapes.append({
        'type': 'line',
        'line':{
            'dash': 'dash',
            'width': 1,
        },
        'xref': 'x',
        'yref': 'y',
        'x0': jump,
        'y0': max_value,
        'x1': jump,
        'y1': min_value
    })

# Set the graph line
data = [
    go.Scatter(
        x = list(range(len(rmsd_data))),
        y = rmsd_data,
    ),
]
wt_fig = go.Figure({
    "data": data,
    "layout": go.Layout(
        title={
            'text': 'RMSD analysis',
        },
        xaxis={'title': {'text': 'Frames'}},
        yaxis={'title': {'text': 'RMSD values'}},
        shapes=shapes
    )
})
wt_fig.show()